In [2]:
!pip install streamlit
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.8 MB/s eta 0:00:00


In [3]:
def load_and_unify():
    google = pd.read_csv("/content/google_ads-sheet1-sourcetable.csv")
    linkedin = pd.read_csv("/content/linked_in_campaigns-sheet1-sourcetable.csv")
    meta = pd.read_csv("/content/meta_ads_campigns-sheet1-sourcetable.csv")

    # Clean money
    google['Cost'] = google['Cost'].replace(r'[$,]', '', regex=True).astype(float)
    linkedin['Total Spent (USD)'] = linkedin['Total Spent (USD)'].replace(r'[$,]', '', regex=True).astype(float)
    meta['Amount spent'] = meta['Amount spent'].replace(r'[$,]', '', regex=True).astype(float)
    if 'Purchase value' in meta.columns:
        meta['Purchase value'] = meta['Purchase value'].replace(r'[$,]', '', regex=True).astype(float)

    # Standardize
    google = google.rename(columns={'Day':'Date','Campaign':'Campaign Name','Country':'Region','Cost':'Spend'})
    google['Platform'] = 'Google Ads'; google['Revenue'] = 0

    linkedin = linkedin.rename(columns={'Day':'Date','Total Spent (USD)':'Spend'})
    if 'Conversions' in linkedin.columns: linkedin.drop(columns=['Conversions'],inplace=True)
    linkedin = linkedin.rename(columns={'Leads':'Conversions'})
    linkedin['Platform']='LinkedIn'; linkedin['Revenue']=0; linkedin['Region']='Global'; linkedin['Device']='Unknown'

    meta = meta.rename(columns={'Campaign name':'Campaign Name','Amount spent':'Spend','Purchases':'Conversions','Purchase value':'Revenue'})
    meta['Platform']='Meta'; meta['Region']='Unknown'; meta['Device']='Mixed'

    cols = ['Date','Platform','Campaign Name','Region','Device','Impressions','Clicks','Spend','Conversions','Revenue']
    google = google.reindex(columns=cols, fill_value=0)
    linkedin = linkedin.reindex(columns=cols, fill_value=0)
    meta = meta.reindex(columns=cols, fill_value=0)

    df = pd.concat([google, linkedin, meta], ignore_index=True)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['CTR'] = df['Clicks']/df['Impressions'].replace(0,np.nan)
    df['CPC'] = df['Spend']/df['Clicks'].replace(0,np.nan)
    df['ROAS'] = df['Revenue']/df['Spend'].replace(0,np.nan)
    df['ROAS'] = df['ROAS'].fillna(0).replace([np.inf,-np.inf],0)
    return df

st.set_page_config(page_title="Ad Profit AI", layout="wide")
st.title("Ad Profit AI – Instant Money Leaks Detector")
st.markdown("**Google Ads + LinkedIn + Meta → One-click profit analysis**")

@st.cache_data
def analyze():
    df = load_and_unify()
    X = df[['Spend','Conversions','Revenue','ROAS','CTR']].fillna(0)
    X_scaled = StandardScaler().fit_transform(X)
    df['Segment'] = KMeans(n_clusters=4, random_state=42, n_init=10).fit_predict(X_scaled)
    df['Anomaly'] = IsolationForest(contamination=0.1, random_state=42).fit_predict(X_scaled)
    return df

df = analyze()

col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Spend", f"${df['Spend'].sum():,.0f}")
col2.metric("Revenue", f"${df['Revenue'].sum():,.0f}")
col3.metric("Profit", f"${df['Revenue'].sum()-df['Spend'].sum():,.0f}")
col4.metric("ROAS", f"{(df['Revenue'].sum()/df['Spend'].sum() if df['Spend'].sum()>0 else 0):.2f}x")

bleeding = df[(df['Spend']>200) & (df['Conversions']==0) & (df['Revenue']==0)]
if len(bleeding)>0:
    st.error(f"URGENT: Pause {len(bleeding)} campaigns → ${bleeding['Spend'].sum():,.0f} wasted!")

st.success("Your AI Profit Analyzer is LIVE!")

fig = px.scatter(df, x="Spend", y="Revenue", color="Platform", size="ROAS", hover_data=["Campaign Name"],
                 title="Where Your Ad Money Actually Makes Profit")
st.plotly_chart(fig, use_container_width=True)

st.balloons()

2025-11-26 10:26:27.368 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 10:26:27.369 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 10:26:27.604 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-26 10:26:27.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 10:26:27.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 10:26:27.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-26 10:26:27.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator()